In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os
import numpy as np
import pyspark.sql.functions as f
import subprocess
from pyspark.sql.functions import min, max, col
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import udf

In [2]:
sc = SparkContext()
sqlcontext = SQLContext(sc)

In [3]:
dir_in = "/user/hadoop/data/landValue"
args = "hdfs dfs -ls "+dir_in+" | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.decode('utf-8').split()
print(all_dart_dirs)

['/user/hadoop/data/landValue/landValue_14.csv', '/user/hadoop/data/landValue/landValue_15.csv', '/user/hadoop/data/landValue/landValue_16.csv', '/user/hadoop/data/landValue/landValue_17.csv', '/user/hadoop/data/landValue/landValue_18.csv']


In [4]:
df = sqlcontext.read.format("com.databricks.spark.csv")\
    .option("header", "true").option("inferSchema", "true").load(all_dart_dirs)

In [5]:
df.show()
df.count()

+-------------------+---------+----+
|           CityCode|    Price|Year|
+-------------------+---------+----+
|1111018200101340012| 601600.0|2014|
|1111018200101350000|2292000.0|2014|
|1111018200101360001| 317100.0|2014|
|1111018200101360002|1867000.0|2014|
|1111018200101360003| 594000.0|2014|
|1111018200101360004|2182000.0|2014|
|1111018200101360005| 577500.0|2014|
|1111018200101360006| 577500.0|2014|
|1111018200101360007| 577500.0|2014|
|1111018200101360008|1926000.0|2014|
|1111018200101360009| 339300.0|2014|
|1111018200101360010|1926000.0|2014|
|1111018200101360011|1926000.0|2014|
|1111018200101360012|1926000.0|2014|
|1111018200101360013|1955000.0|2014|
|1111018200101360014|1867000.0|2014|
|1111018200101360015|1867000.0|2014|
|1111018200101360016|1867000.0|2014|
|1111018200101360017|1867000.0|2014|
|1111018200101360018|1867000.0|2014|
+-------------------+---------+----+
only showing top 20 rows



4669061

In [6]:
total_df = df.groupBy("CityCode").agg(f.mean('Price'))
total_df.show()

+-------------------+----------+
|           CityCode|avg(Price)|
+-------------------+----------+
|1111018200101380019|  176140.0|
|1111018300104760016| 2036800.0|
|1111018400102510018|  448800.0|
|1111018300104460011| 1857200.0|
|1111018400101940005| 1338000.0|
|1111018400101980001|  441540.0|
|1111018200101270003| 1328200.0|
|1111018100100160013| 4043000.0|
|1111017900100420010| 4205400.0|
|1111018200100810001|  353360.0|
|1111018200102300039| 1862000.0|
|1111018100100010117| 2566000.0|
|1111017500102400016| 5128800.0|
|1111016700101360002| 5497000.0|
|1111018600101600003| 1357800.0|
|1111017400106270074|  725340.0|
|1111017300100011083| 1632200.0|
|1111016400102390020| 4228000.0|
|1111015900102530000| 3090000.0|
|1111016700100310025| 6852800.0|
+-------------------+----------+
only showing top 20 rows



In [7]:
total_df.count()

967473

In [8]:
total_df.write.csv("merge_LandValue")

AnalysisException: 'path hdfs://master:9000/user/hadoop/merge_LandValue already exists.;'

In [ ]:
# landValue file merge
sc.textFile("hdfs://master:9000/user/hadoop/merge_LandValue/part*").coalesce(1).saveAsTextFile("hdfs://master:9000/user/hadoop/merge_LandValue_")

In [9]:
h2b_df = sqlcontext.read.option("header","true").csv("data/h2b.csv")

In [10]:
h2b_df.show()
h2b_df.count()

+----------+-----+----+-----+----------+-----+--------+----+
|     행정동코드|  시도명|시군구명| 읍면동명|     법정동코드|  동리명|    생성일자|말소일자|
+----------+-----+----+-----+----------+-----+--------+----+
|1100000000|서울특별시|null| null|1100000000|서울특별시|19880423|null|
|1111000000|서울특별시| 종로구| null|1111000000|  종로구|19880423|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010100|  청운동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010200|  신교동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010300|  궁정동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010400|  효자동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010500|  창성동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010800|  통인동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111010900|  누상동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111011000|  누하동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111011100|  옥인동|20081101|null|
|1111051500|서울특별시| 종로구|청운효자동|1111011900|  세종로|20081101|null|
|1111053000|서울특별시| 종로구|  사직동|1111010600|  통의동|19880423|null|
|1111053000|서울특별시| 종로구| 

21697

In [11]:
test_df = h2b_df.groupBy("법정동코드").agg(f.collect_set('동리명'), f.collect_set('시군구명'))
test_df.show()
test_df.count()

+----------+----------------+-----------------+
|     법정동코드|collect_set(동리명)|collect_set(시군구명)|
+----------+----------------+-----------------+
|1144010400|           [도화동]|            [마포구]|
|2711013800|         [계산동1가]|             [중구]|
|2717010900|         [원대동3가]|             [서구]|
|2820010500|           [서창동]|            [남동구]|
|3014011400|           [태평동]|             [중구]|
|3171025926|           [두산리]|            [울주군]|
|3611034022|           [발산리]|               []|
|4122025925|           [후사리]|            [평택시]|
|4150036028|           [소고리]|            [이천시]|
|4150037028|           [암산리]|            [이천시]|
|4155033027|           [동촌리]|            [안성시]|
|4159013700|            [송동]|            [화성시]|
|4161033026|           [도웅리]|            [광주시]|
|4180033022|           [백령리]|            [연천군]|
|4180035026|           [동중리]|            [연천군]|
|4215035026|           [남양리]|            [강릉시]|
|4221000000|           [속초시]|            [속초시]|
|4278036030|           [노동리]|           

20560

In [13]:
_joined = test_df.join(h2b_df, test_df.법정동코드 == h2b_df.법정동코드, how='inner')
_joined.count()

21697

In [1]:
t = total_df.withColumn('bub', regexp_extract('CityCode', '(^[0-9]{10})', 1))
t.show()
t.count()

NameError: name 'total_df' is not defined

In [15]:
joined = t.join(test_df, t.bub == test_df.법정동코드, how='inner')
joined.count()

967473

In [2]:
join_udf = udf(lambda x: ",".join(x))
s_joined = joined.withColumn("collect_set(동리명)", join_udf(col("collect_set(동리명)")))\
        .withColumn("collect_set(시군구명)", join_udf(col("collect_set(시군구명)")))

NameError: name 'udf' is not defined

In [ ]:
joined.show()

In [17]:
t = s_joined.withColumn('mountain', regexp_extract('CityCode', '^[0-9]{10}([0-9])', 1)).\
            withColumn('bon', regexp_extract('CityCode', '^[0-9]{11}([0-9]{4})', 1)).\
            withColumn('bu', regexp_extract('CityCode', '^[0-9]{15}([0-9]{4})', 1))

In [18]:
from pyspark.sql.types import IntegerType
in_df = t.withColumn("bon", t["bon"].cast(IntegerType()))
in_df = in_df.withColumn("bu", t["bu"].cast(IntegerType()))

In [ ]:
in_df.show()

In [ ]:
t.show()

In [19]:
_t = in_df.withColumn('moun', f.when(f.col('mountain') > '1', "산").otherwise(""))

In [23]:
result_df = _t.withColumn("Address", concat( col("collect_set(시군구명)"), lit(" "), col("collect_set(동리명)"), lit(" ") \
                            , col('moun') ,col('bon') )).select('Address', 'avg(Price)')

In [24]:
test_df = result_df.groupBy("Address").agg(f.mean('avg(Price)'))
test_df.show()
test_df.count()

+--------------+--------------------+
|       Address|     avg(avg(Price))|
+--------------+--------------------+
|     중구 남창동 60|1.4366553846153846E7|
|   중구 을지로6가 17|1.2686191048034934E7|
|   용산구 한남동 620|   4557897.142857143|
|  성동구 성수동1가 98|  3472657.1428571427|
|   광진구 중곡동 129|           2479735.0|
|   성북구 정릉동 132|  2463352.5714285714|
|   성북구 정릉동 581|  1723336.6666666667|
|   강북구 수유동 538|  1303412.9166666667|
| 서대문구 북아현동 173|  3435000.0000000005|
|   마포구 공덕동 464|           2331060.0|
|   강서구 염창동 277|  3713199.6923076925|
|   구로구 구로동 448|   1927122.528735632|
|  영등포구 도림동 160|  2614118.1102362205|
|   동작구 흑석동 276|           2115920.0|
|  관악구 신림동 1556|  3620615.0943396227|
|   강동구 상일동 404|           1018135.0|
|서대문구 북아현동 1012|           2842900.0|
|    종로구 봉익동 38|   5527752.380952381|
|    중구 신당동 130|           6772925.0|
| 성동구 성수동2가 333|   2917654.255319149|
+--------------+--------------------+
only showing top 20 rows



106582

In [21]:
result_df.count()

967473

In [ ]:
result_df.write.csv("Address_")

In [ ]:
sc.textFile("hdfs://master:9000/user/hadoop/Address_/part*").coalesce(1).saveAsTextFile("hdfs://master:9000/user/hadoop/Address__")